Python Blood Cell Identification using Image Processing
This code implements suitable image segmentation and feature extraction techniques for blood cell identification. 

In [2]:
import numpy as np
import cv2 # allows us to manipulate images 
import matplotlib.pyplot as plt


In [3]:
# Reading in cell image
image = cv2.imread("c120.png")

In [4]:
# converting image to gray scale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imwrite("gray.png", gray)


True

In [5]:
# Applying median filter for smoothing 
blur_median_img = cv2.medianBlur(gray,5)
cv2.imwrite("median_blur_img.png", blur_median_img)


True

In [6]:
# Apply Gaussian filter for smoothing
blur_gaus_img = cv2.GaussianBlur(gray,(9,9),0)
cv2.imwrite("gaussian_blur.png", blur_gaus_img)

True

In [7]:
# apply histogram equalization
histo_norm = cv2.equalizeHist(gray)
cv2.imwrite("histoNorm.png",histo_norm)

True

In [8]:
# Create a CLAHE Object (Contrast limited adaptive histogram equalization) Image
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
claheNorm = clahe.apply(gray)
cv2.imwrite("claheNorm.png", claheNorm)

True

In [9]:
# Contrast stretching, create a function to map each intensity level to output intensity level
def pixelVal(pix,r1,s1,r2,s2):
    if(0<-pix and pix <= r1):
        return (s1/r1)*pix
    elif (r1<pix and pix <=r2):
        return (((s2-s1)/(r2-r1))* (pix-r1)*s1)
    else:
        return ((255-s2)/(255-r2)) * (pix-r2)+s2


In [10]:
# Define parameters 
r1 = 70
s1 = 0
r2 = 200
s2 = 255

In [11]:
# vectorize the function to apply it to each value in the numpy array
pixelVal_vec = np.vectorize(pixelVal)


In [12]:
# Apply contrast stretching
contrast_stretched = pixelVal_vec(gray,r1,s1,r2,s2)
contrast_stretched_blurM = pixelVal_vec(blur_median_img,r1,s1,r2,s2)

cv2.imwrite("contrast_stretched.png", contrast_stretched)
cv2.imwrite("contrast_stretched_blurM.png", contrast_stretched_blurM)
# This provides us an iamge with just the cells isolated to work with


/var/folders/21/b29gvtpd4cj9j513pw_p472w0000gn/T/ipykernel_87179/496490216.py:3: RuntimeWarning: overflow encountered in scalar negative
  if(0<-pix and pix <= r1):
[ WARN:0@0.408] global loadsave.cpp:1063 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


True

In [13]:
# Edge detecting using canny edge detector
edge = cv2.Canny(gray, 100, 200)
cv2.imwrite("edge.png", edge)

edgeG = cv2.Canny(blur_gaus_img, 100, 200)
cv2.imwrite("edgeG.png",edgeG)

edgeM = cv2.Canny(blur_median_img, 100, 200)
cv2.imwrite("edgeM.png", edgeM)

True

In [14]:
# read enhanced image 
img = cv2.imread('c120.png', 0)

# morphological operations
kernel = np.ones((5, 5), np.uint8)
dilation = cv2.dilate(img, kernel, iterations = 1)
closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

# Adaptive thresholding on mean and gaussian filter
th2 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, \
            cv2.THRESH_BINARY, 11, 2)
th3 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, \
            cv2.THRESH_BINARY, 11, 2)
# Otsu's thresholding
ret4, th4 = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

In [15]:

# Initialize the list
Cell_count, x_count, y_count = [], [], []

# read original image, to display the circle and center detection  
display = cv2.imread("c120.png")

# hough transform with modified circular parameters
circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 1.2, 20, 
                           param1 = 50, param2 = 28, minRadius = 1, maxRadius = 20)

stuck in a loop, need to fix still

In [ ]:
# circle detection and labeling using hough transformation 
if circles is not None:
        # convert the (x, y) coordinates and radius of the circles to integers
        circles = np.round(circles[0, :]).astype("int")

        # loop over the (x, y) coordinates and radius of the circles
        for (x, y, r) in circles:

                cv2.circle(display, (x, y), r, (0, 255, 0), 2)
                cv2.rectangle(display, (x - 2, y - 2), 
                              (x + 2, y + 2), (0, 128, 255), -1)
                Cell_count.append(r)
                x_count.append(x)
                y_count.append(y)
                
        # show the output image
        cv2.imshow("gray", display)
        cv2.waitKey(0)

# display the count of white blood cells 
print(len(Cell_count))
# Total number of radius
print(Cell_count) 
# X co-ordinate of circle
print(x_count)     
# Y co-ordinate of circle
print(y_count)